In [99]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime, timedelta
#para request
import requests
#parametros
import parametros
import time

## Leyendo datos del indice ohmyfi-detalleconexiones

In [100]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

### Definiendo fechas para rangos de consultas

In [101]:
now = datetime.now()
new_date = now - timedelta(days=6)
ahora = str(now.strftime("%Y/%m/%d %H:%M"))
fecha_6 = str(new_date.strftime("%Y/%m/%d %H:%M"))

## tomando fecha mas reciente del indice

In [102]:
total_docs = 0
try:
    response = es.search(
        index=parametros.ohmyfi_d_c_index,
        body={"aggs" : {
                   "max_date": {"max": {"field": "fechahora", "format": "yyyy/MM/dd HH:mm"}}
                }
             },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["aggregations"]
    fecha_max = response["aggregations"]["max_date"]['value_as_string']
    h_inicio = response["aggregations"]["max_date"]['value_as_string']
    h_fin = (datetime.strptime(h_inicio, '%Y/%m/%d %H:%M')+timedelta(hours=1)).strftime("%Y/%m/%d %H:%M")
except:
    fecha_max = fecha_6
    h_inicio = fecha_6
    h_fin = str((new_date+timedelta(hours=1)).strftime("%Y/%m/%d %H:%M"))

print("Fecha maxima en indice:", fecha_max,"| Fecha inicio:", h_inicio, " | Fecha fin:", h_fin)

Fecha maxima en indice: 2021/10/14 13:30 | Fecha inicio: 2021/10/14 13:30  | Fecha fin: 2021/10/14 14:30


In [103]:
h_inicio, h_fin, ahora

('2021/10/14 13:30', '2021/10/14 14:30', '2021/10/14 13:31')

## Leyendo la APi detalleconexiones

In [104]:
url2 = parametros.url_ohmyfi +'?/detalleconexiones/=&apiKey=' + parametros.ohmyfi_api_key 

In [105]:
datos_api = pd.DataFrame(columns=['mac_usuario','nombre_completo','apellidos','correo','documento','tipodoc'
                                      ,'celular','genero','fecha_nacimiento','dispositivo','sistema_operativo'
                                      ,'fecha','fechahora','lugar','tipo','id_lugar','lugar_cod'])
iterar = 0
while (h_inicio < ahora) & (iterar < 27):
    iterar = iterar + 1
    if h_fin[11:16] == '00:59':
        h_inicio = h_fin[0:11] + '00:00'
    elif h_inicio[8:10] != h_fin[8:10]:
        h_inicio = h_inicio[0:11] + '23:00'
        h_fin = h_inicio[0:11] + '23:59'
    url = url2 + '&inicio=' + str(h_inicio) + '&final=' + str(h_fin) 
    r = requests.get(url, timeout=15)
    if r.status_code == 200:
        res = json.loads(r.text)
        print(h_inicio)
        datos_api = datos_api.append(res, ignore_index=True)
        #datos_api.drop_duplicates(inplace=True)
    else:
        print("Error de request:",r.status_code)
        if r.status_code != 204:
            print("Error:",r.status_code,"URL:",url)
        #exit()
    h_inicio = h_fin
    h_fin = (datetime.strptime(h_inicio, '%Y/%m/%d %H:%M') + timedelta(hours=1)).strftime("%Y/%m/%d %H:%M")
    time.sleep(3)

2021/10/14 13:30


In [106]:
url

'https://www.ohmyfi.com/ApiOMF?/detalleconexiones/=&apiKey=okMOpLAkiYpafQKXhXirwUys&inicio=2021/10/14 13:30&final=2021/10/14 14:30'

In [107]:
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora
0,FC-42-03-ED-E3-78,Luis Carlos,Vuelvas martines,None,1192891438,CC,None,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:00.221445-05,EL PORVENIR_TURBO,True,8988,21985-VZVF899,13:30:00
1,BE-95-6F-F1-B7-A3,Yissela,Ibarra,yisselithamerchan@08hotmail.com,1234645605,CC,3133703306,f,None,Smartphone,AndroidOS,2021-10-14,2021-10-14 13:30:00.937029-05,RINCON ANCHIQUE_NATAGAIMA,False,6060,48849-ZGYO533,13:30:00
2,18-65-71-CC-B4-02,Eder,Reales,scorpio6419@live.com,8635137,CC,3148934146,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:01.022195-05,CALLE 7 409_CANDELARIA,True,7452,22486-ZZZY553,13:30:00
3,EC-AA-25-02-F2-FC,Sirley,López,LopezSirley83@Gmail.Com,1039092360,TI,3234205523,f,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:01.511995-05,VDA GARITON_NECOCLI,False,5908,20693-ZGYO169,13:30:00
4,68-A3-C4-E9-20-12,dayanis,mercado,None,1100623946,TI,3217880378,f,None,PC,Windows,2021-10-14,2021-10-14 13:30:01.605875-05,"PICHILIN,MORROA",False,6498,47368-ZZZY269,13:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1C-23-2C-F2-63-B3,Jaider,Trillos,jtrillosm25@gmail.com,1062066705,TI,3172831791,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:31:37.517654-05,EL PLAN_LA JAGUA DEL PILAR,False,6776,37742-ZGYO308,13:30:00
296,14-96-E5-F4-61-35,Nohemi,Gonzalez urango,None,586528283,TI,3107489676,f,None,Smartphone,Android,2021-10-14,2021-10-14 13:31:38.211317-05,CABUYA_SAN_CARLOS_CORDOBA,True,6370,32404-ZGYO453,13:30:00
297,40-A3-CC-2F-1E-A3,cristian david,gonzalez murillo,cristiandavidgonzalez852@gmail.con,1038646906,TI,3202916290,m,None,PC,Windows,2021-10-14,2021-10-14 13:31:38.279554-05,LOS CASTILLOS_PALMITO,False,5979,47409-ZGYO725,13:30:00
298,56-79-09-14-26-D9,Andres,Quintero,None,1037236185,CC,573234962775,m,None,Smartphone,AndroidOS,2021-10-14,2021-10-14 13:31:38.449676-05,VDA LA PENA_GUATAPE,True,6738,20224-ZZZY299,13:30:00


In [108]:
datos_api.to_excel("detalles_crudo_14_10_21.xlsx")

### Descartando datos de la API que ya están en el indice

Si el dataframe datos_api no tiene datos, cae en el except.

In [109]:
h_inicio, h_fin, ahora
#datos_api2 = datos_api

('2021/10/14 14:30', '2021/10/14 15:30', '2021/10/14 13:31')

In [110]:
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora
0,FC-42-03-ED-E3-78,Luis Carlos,Vuelvas martines,None,1192891438,CC,None,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:00.221445-05,EL PORVENIR_TURBO,True,8988,21985-VZVF899,13:30:00
1,BE-95-6F-F1-B7-A3,Yissela,Ibarra,yisselithamerchan@08hotmail.com,1234645605,CC,3133703306,f,None,Smartphone,AndroidOS,2021-10-14,2021-10-14 13:30:00.937029-05,RINCON ANCHIQUE_NATAGAIMA,False,6060,48849-ZGYO533,13:30:00
2,18-65-71-CC-B4-02,Eder,Reales,scorpio6419@live.com,8635137,CC,3148934146,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:01.022195-05,CALLE 7 409_CANDELARIA,True,7452,22486-ZZZY553,13:30:00
3,EC-AA-25-02-F2-FC,Sirley,López,LopezSirley83@Gmail.Com,1039092360,TI,3234205523,f,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:01.511995-05,VDA GARITON_NECOCLI,False,5908,20693-ZGYO169,13:30:00
4,68-A3-C4-E9-20-12,dayanis,mercado,None,1100623946,TI,3217880378,f,None,PC,Windows,2021-10-14,2021-10-14 13:30:01.605875-05,"PICHILIN,MORROA",False,6498,47368-ZZZY269,13:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1C-23-2C-F2-63-B3,Jaider,Trillos,jtrillosm25@gmail.com,1062066705,TI,3172831791,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:31:37.517654-05,EL PLAN_LA JAGUA DEL PILAR,False,6776,37742-ZGYO308,13:30:00
296,14-96-E5-F4-61-35,Nohemi,Gonzalez urango,None,586528283,TI,3107489676,f,None,Smartphone,Android,2021-10-14,2021-10-14 13:31:38.211317-05,CABUYA_SAN_CARLOS_CORDOBA,True,6370,32404-ZGYO453,13:30:00
297,40-A3-CC-2F-1E-A3,cristian david,gonzalez murillo,cristiandavidgonzalez852@gmail.con,1038646906,TI,3202916290,m,None,PC,Windows,2021-10-14,2021-10-14 13:31:38.279554-05,LOS CASTILLOS_PALMITO,False,5979,47409-ZGYO725,13:30:00
298,56-79-09-14-26-D9,Andres,Quintero,None,1037236185,CC,573234962775,m,None,Smartphone,AndroidOS,2021-10-14,2021-10-14 13:31:38.449676-05,VDA LA PENA_GUATAPE,True,6738,20224-ZZZY299,13:30:00


In [111]:
datos_api.drop_duplicates(inplace=True)
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora
0,FC-42-03-ED-E3-78,Luis Carlos,Vuelvas martines,None,1192891438,CC,None,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:00.221445-05,EL PORVENIR_TURBO,True,8988,21985-VZVF899,13:30:00
1,BE-95-6F-F1-B7-A3,Yissela,Ibarra,yisselithamerchan@08hotmail.com,1234645605,CC,3133703306,f,None,Smartphone,AndroidOS,2021-10-14,2021-10-14 13:30:00.937029-05,RINCON ANCHIQUE_NATAGAIMA,False,6060,48849-ZGYO533,13:30:00
2,18-65-71-CC-B4-02,Eder,Reales,scorpio6419@live.com,8635137,CC,3148934146,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:01.022195-05,CALLE 7 409_CANDELARIA,True,7452,22486-ZZZY553,13:30:00
3,EC-AA-25-02-F2-FC,Sirley,López,LopezSirley83@Gmail.Com,1039092360,TI,3234205523,f,None,Smartphone,Android,2021-10-14,2021-10-14 13:30:01.511995-05,VDA GARITON_NECOCLI,False,5908,20693-ZGYO169,13:30:00
4,68-A3-C4-E9-20-12,dayanis,mercado,None,1100623946,TI,3217880378,f,None,PC,Windows,2021-10-14,2021-10-14 13:30:01.605875-05,"PICHILIN,MORROA",False,6498,47368-ZZZY269,13:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1C-23-2C-F2-63-B3,Jaider,Trillos,jtrillosm25@gmail.com,1062066705,TI,3172831791,m,None,Smartphone,Android,2021-10-14,2021-10-14 13:31:37.517654-05,EL PLAN_LA JAGUA DEL PILAR,False,6776,37742-ZGYO308,13:30:00
296,14-96-E5-F4-61-35,Nohemi,Gonzalez urango,None,586528283,TI,3107489676,f,None,Smartphone,Android,2021-10-14,2021-10-14 13:31:38.211317-05,CABUYA_SAN_CARLOS_CORDOBA,True,6370,32404-ZGYO453,13:30:00
297,40-A3-CC-2F-1E-A3,cristian david,gonzalez murillo,cristiandavidgonzalez852@gmail.con,1038646906,TI,3202916290,m,None,PC,Windows,2021-10-14,2021-10-14 13:31:38.279554-05,LOS CASTILLOS_PALMITO,False,5979,47409-ZGYO725,13:30:00
298,56-79-09-14-26-D9,Andres,Quintero,None,1037236185,CC,573234962775,m,None,Smartphone,AndroidOS,2021-10-14,2021-10-14 13:31:38.449676-05,VDA LA PENA_GUATAPE,True,6738,20224-ZZZY299,13:30:00


In [112]:
try:
    datos_api = datos_api.drop_duplicates()
    datos_api.fillna('', inplace=True)
    datos_api['fechahora'] = datos_api["fechahora"].str.split(".", n = 1, expand = True)[0]
    datos_api = datos_api.drop(datos_api[(datos_api['fechahora']<= fecha_max.replace("/","-"))].index)
    datos_api['mac_usuario'] = datos_api['mac_usuario'].str.replace('-',':')
except:
    pass

In [113]:
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora
0,FC:42:03:ED:E3:78,Luis Carlos,Vuelvas martines,,1192891438,CC,,m,,Smartphone,Android,2021-10-14,2021-10-14 13:30:00,EL PORVENIR_TURBO,True,8988,21985-VZVF899,13:30:00
1,BE:95:6F:F1:B7:A3,Yissela,Ibarra,yisselithamerchan@08hotmail.com,1234645605,CC,3133703306,f,,Smartphone,AndroidOS,2021-10-14,2021-10-14 13:30:00,RINCON ANCHIQUE_NATAGAIMA,False,6060,48849-ZGYO533,13:30:00
2,18:65:71:CC:B4:02,Eder,Reales,scorpio6419@live.com,8635137,CC,3148934146,m,,Smartphone,Android,2021-10-14,2021-10-14 13:30:01,CALLE 7 409_CANDELARIA,True,7452,22486-ZZZY553,13:30:00
3,EC:AA:25:02:F2:FC,Sirley,López,LopezSirley83@Gmail.Com,1039092360,TI,3234205523,f,,Smartphone,Android,2021-10-14,2021-10-14 13:30:01,VDA GARITON_NECOCLI,False,5908,20693-ZGYO169,13:30:00
4,68:A3:C4:E9:20:12,dayanis,mercado,,1100623946,TI,3217880378,f,,PC,Windows,2021-10-14,2021-10-14 13:30:01,"PICHILIN,MORROA",False,6498,47368-ZZZY269,13:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1C:23:2C:F2:63:B3,Jaider,Trillos,jtrillosm25@gmail.com,1062066705,TI,3172831791,m,,Smartphone,Android,2021-10-14,2021-10-14 13:31:37,EL PLAN_LA JAGUA DEL PILAR,False,6776,37742-ZGYO308,13:30:00
296,14:96:E5:F4:61:35,Nohemi,Gonzalez urango,,586528283,TI,3107489676,f,,Smartphone,Android,2021-10-14,2021-10-14 13:31:38,CABUYA_SAN_CARLOS_CORDOBA,True,6370,32404-ZGYO453,13:30:00
297,40:A3:CC:2F:1E:A3,cristian david,gonzalez murillo,cristiandavidgonzalez852@gmail.con,1038646906,TI,3202916290,m,,PC,Windows,2021-10-14,2021-10-14 13:31:38,LOS CASTILLOS_PALMITO,False,5979,47409-ZGYO725,13:30:00
298,56:79:09:14:26:D9,Andres,Quintero,,1037236185,CC,573234962775,m,,Smartphone,AndroidOS,2021-10-14,2021-10-14 13:31:38,VDA LA PENA_GUATAPE,True,6738,20224-ZZZY299,13:30:00


### fecha_control se usará para la agrupación cada 10 minutos

In [120]:
datos_api['fecha_control'] = datos_api["fechahora"].str[0:-4] + '0:00'

In [ ]:
datos_api

### Insertando datos en el indice
* Cambio quitar fecha (Solicitado por Napoleon [GM])


In [118]:
indice = 'ohmyfi-detalles_pruingesta14_10_21'

In [119]:
use_these_keys = ['mac_usuario','nombre_completo','apellidos','correo','documento','tipodoc','celular'
                  ,'genero','fecha_nacimiento','dispositivo','sistema_operativo','fecha','fechahora','fecha_control'
                  ,'lugar','tipo','id_lugar','lugar_cod','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

timestamp = datetime.now()
datos_api['@timestamp'] = timestamp.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_api))
print("Fecha: ",now,"- Documentos insertados: ",salida[0])

BulkIndexError: ('295 document(s) failed to index.', [{'index': {'_index': 'ohmyfi-detalles_pruingesta', '_type': '_doc', '_id': 'k4odgHwBaOZ5dxm4-v8O', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': "failed to parse field [fecha_nacimiento] of type [date] in document with id 'k4odgHwBaOZ5dxm4-v8O'. Preview of field's value: ''", 'caused_by': {'type': 'illegal_argument_exception', 'reason': 'cannot parse empty date'}}, 'data': {'mac_usuario': 'FC:42:03:ED:E3:78', 'nombre_completo': 'Luis Carlos', 'apellidos': 'Vuelvas martines', 'correo': '', 'documento': '1192891438', 'tipodoc': 'CC', 'celular': '', 'genero': 'm', 'fecha_nacimiento': '', 'dispositivo': 'Smartphone', 'sistema_operativo': 'Android', 'fecha': '2021-10-14', 'fechahora': '2021-10-14 13:30:00', 'fecha_control': '2021-10-14 13:30:00', 'lugar': 'EL PORVENIR_TURBO ', 'tipo': True, 'id_lugar': 8988, 'lugar_cod': '21985-VZVF899', '@timestamp': '2021-10-14T13:43:40.631173'}}}, {'index': {'_index': 'ohmyfi-